In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IN_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
260,3269267107,IN,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,IN,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,IN,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
509,8772907000,IN,"777 South Main Street Suite 100, CLINTON, IN",39.653545,-87.399108,NaN
581,5283147003,IN,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
...,...,...,...,...,...,...
15662,4653648305,IN,"200 S Parker Dr, Ashley, IN",41.525697,-85.075028,NaN
15957,6045077005,IN,"10525 W US Hwy 30 Bldg 11 B, WANATAH, IN",40.778572,-98.740866,NaN
15971,8056608402,IN,"69 E Garner Rd, Brownsburg, IN",40.622322,-79.139510,NaN
16444,8460197100,IN,"600 W Maple Street PO Box 97, WATERLOO, IN",40.613804,-79.150734,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('18')] #IN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/65 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

65


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9213/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
260,3269267107,IN,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,[]
364,7007407110,IN,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,[]
412,6311077009,IN,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,[]
509,8772907000,IN,"777 South Main Street Suite 100, CLINTON, IN",39.653545,-87.399108,[181650205006012]
581,5283147003,IN,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,[]
...,...,...,...,...,...,...
15662,4653648305,IN,"200 S Parker Dr, Ashley, IN",41.525697,-85.075028,[180330201001006]
15957,6045077005,IN,"10525 W US Hwy 30 Bldg 11 B, WANATAH, IN",40.778572,-98.740866,[]
15971,8056608402,IN,"69 E Garner Rd, Brownsburg, IN",40.622322,-79.139510,[]
16444,8460197100,IN,"600 W Maple Street PO Box 97, WATERLOO, IN",40.613804,-79.150734,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_9213/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
260,3269267107,IN,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,IN,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,IN,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
509,8772907000,IN,"777 South Main Street Suite 100, CLINTON, IN",39.653545,-87.399108,181650205006012
581,5283147003,IN,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
...,...,...,...,...,...,...
15662,4653648305,IN,"200 S Parker Dr, Ashley, IN",41.525697,-85.075028,180330201001006
15957,6045077005,IN,"10525 W US Hwy 30 Bldg 11 B, WANATAH, IN",40.778572,-98.740866,NaN
15971,8056608402,IN,"69 E Garner Rd, Brownsburg, IN",40.622322,-79.139510,NaN
16444,8460197100,IN,"600 W Maple Street PO Box 97, WATERLOO, IN",40.613804,-79.150734,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
260,3269267107,IN,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,IN,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,IN,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
581,5283147003,IN,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
1022,7535037110,IN,"701 WATER ST, FREMONT, IN",40.624320,-79.153366,NaN
1256,6612367104,IN,"8735 Hwy 66 E, LAMAR, IN",36.148057,-95.878467,NaN
1370,6565827009,IN,"P.O Box 67 425 Main Street NE, PALMYRA, IN",40.758663,-73.953093,NaN
2091,3646578403,IN,"5401 W Airport Rd, Gary, IN",40.653263,-79.039024,NaN
2276,1201748502,IN,"111 E Innovation Blvd, Syracuse, IN",40.621834,-79.139520,NaN
2584,3344197100,IN,"5401 AIRPORT RD, GARY, IN",40.653255,-79.039030,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
260,3269267107,NaN
364,7007407110,NaN
412,6311077009,NaN
509,8772907000,181650205006012
581,5283147003,NaN
...,...,...
15662,4653648305,180330201001006
15957,6045077005,NaN
15971,8056608402,NaN
16444,8460197100,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IN_fips_scraped.csv") 
fips_check = fips_check.drop(["Unnamed: 0"], axis =1) 
fips_check.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14,NaN
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14,NaN
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14,NaN
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14,NaN
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14,NaN
...,...,...,...,...,...,...,...
16589,9384688310,IN,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,1.809731e+14,NaN
16590,9490728302,IN,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,1.809738e+14,NaN
16591,9571197108,IN,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,1.801795e+14,NaN
16592,9715977705,IN,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,1.805711e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,1.808902e+14,NaN,180890204001028.0
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,1.809739e+14,NaN,180973910011101.0
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,1.809731e+14,NaN,180973102011027.0
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,1.808901e+14,NaN,180890103023014.0
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,1.803900e+14,NaN,180390013011018.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1266777200,IN,"1412 150TH ST, HAMMOND, IN",41.623110,-87.493830,180890204001028.0
1,2313017204,IN,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.160000,180973910011101.0
2,2341197209,IN,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.252910,180973102011027.0
3,3129227104,IN,"7100 W 21st Ave, GARY, IN",41.580840,-87.424965,180890103023014.0
4,5372347006,IN,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.015230,180390013011018.0
...,...,...,...,...,...,...
16589,9384688310,IN,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.240930,180973101051006.0
16590,9490728302,IN,"1818 Main St, Beech Grove, IN",39.721546,-86.103130,180973804042001.0
16591,9571197108,IN,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.225180,180179517001009.0
16592,9715977705,IN,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,180571110033009.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
260,3269267107,IN,"1200 W Washington Street 0.0, Indianapolis, IN",40.619286,-79.163780,NaN
364,7007407110,IN,"9269 E STATE ROAD 48, MILAN, IN",45.464204,9.189982,NaN
412,6311077009,IN,"719 FRONT ST, AKRON, IN",41.117705,-81.493627,NaN
581,5283147003,IN,"PO Box 493, BRAZIL, IN",-14.235004,-51.925280,NaN
1022,7535037110,IN,"701 WATER ST, FREMONT, IN",40.624320,-79.153366,NaN
1256,6612367104,IN,"8735 Hwy 66 E, LAMAR, IN",36.148057,-95.878467,NaN
1370,6565827009,IN,"P.O Box 67 425 Main Street NE, PALMYRA, IN",40.758663,-73.953093,NaN
2091,3646578403,IN,"5401 W Airport Rd, Gary, IN",40.653263,-79.039024,NaN
2276,1201748502,IN,"111 E Innovation Blvd, Syracuse, IN",40.621834,-79.139520,NaN
2584,3344197100,IN,"5401 AIRPORT RD, GARY, IN",40.653255,-79.039030,NaN


Convert to File

In [19]:
fips_IN = fips_merge_drop

In [20]:
fips_IN.to_csv("../../../data/state_data/geo/geo_fips/23/IN_fips_scraped.csv") 